In [1]:
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
tf.reset_default_graph()

In [6]:
tf.set_random_seed(8)

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data',one_hot='True')

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [11]:
trainX=mnist.train.images
trainY=mnist.train.labels

In [12]:
testX=mnist.test.images
testY=mnist.test.labels

In [13]:
trainX.shape

(55000, 784)

In [18]:
logs_path='/tmp/dnn/relu/v6'
learning_rate=0.003
n_features=trainX.shape[1]
n_classes=trainY.shape[1]
model_name= 'mnist.ckpt'
training_epochs=60
batch_size=100
K=200
L=200
M=60
N=30

training_keep=0.75
test_keep=1

In [19]:
with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,shape=[None,n_features],name='input')
    y_=tf.placeholder(tf.float32,shape=[None,n_classes],name='output')
    pkeep=tf.placeholder(tf.float32)

In [22]:
with tf.name_scope('layer1'):
    W1=tf.Variable(tf.truncated_normal([n_features,K],stddev=0.1))
    b1=tf.Variable(tf.zeros([K])) 
    Y1=tf.nn.relu(tf.add(tf.matmul(x,W1),b1))        

In [23]:
with tf.name_scope('layer2'):
    W2=tf.Variable(tf.truncated_normal([K,L],stddev=0.1))
    b2=tf.Variable(tf.zeros([L])) 
    Y2=tf.nn.relu(tf.add(tf.matmul(Y1,W2),b2))        

In [24]:
with tf.name_scope('layer3'):
    W3=tf.Variable(tf.truncated_normal([L,M],stddev=0.1))
    b3=tf.Variable(tf.zeros([M])) 
    Y3=tf.nn.relu(tf.add(tf.matmul(Y2,W3),b3))        

In [25]:
with tf.name_scope('layer4'):
    W4=tf.Variable(tf.truncated_normal([M,N],stddev=0.1))
    b4=tf.Variable(tf.zeros([N])) 
    Y4=tf.nn.relu(tf.add(tf.matmul(Y3,W4),b4)) 
    Y4d=tf.nn.dropout(Y4,pkeep)

In [26]:
# Prediction
with tf.name_scope('output'):
    W5=tf.Variable(tf.truncated_normal([N,n_classes],stddev=0.1))
    b5=tf.Variable(tf.zeros([n_classes]))
    Ylogits=tf.matmul(Y4d,W5)+b5
    y=tf.nn.softmax(tf.add(tf.matmul(Y4d,W5),b5))

In [29]:
# Loss
with tf.name_scope('loss'):
    cross_entropy=tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits,labels=y_)
    cross_entropy=tf.reduce_mean(cross_entropy)*100

In [30]:
# Optimizer
with tf.name_scope('train'):
    train_op=tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [31]:
# Accuracy
with tf.name_scope('accuracy'):
    prediction=tf.argmax(y,1,name='prediction')
    correct_prediction=tf.equal(prediction,tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name='accuracy')

In [33]:
# Loss and accuracy logging
training_loss=tf.summary.scalar('training_loss',cross_entropy)
training_accuracy=tf.summary.scalar('training_accuracy',accuracy)
test_loss=tf.summary.scalar('test_loss',cross_entropy)
test_accuracy=tf.summary.scalar('test_accuracy',accuracy)

In [34]:
# Create saver to save the model
saver=tf.train.Saver()

In [39]:
# execute the graph now
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    writer=tf.summary.FileWriter(logs_path,graph=tf.get_default_graph())
    
    for epoch in range(training_epochs):
        
        batch_count=int(trainX.shape[0]/batch_size)
        for i in range(batch_count):
            batch_x=trainX[i*batch_size:i*batch_size+batch_size]
            batch_y=trainY[i*batch_size:i*batch_size+batch_size]
            
            _,acc,loss=sess.run([train_op,training_accuracy,training_loss],feed_dict={x:batch_x,y_:batch_y,pkeep:training_keep})
            writer.add_summary(loss,epoch * batch_count + i)
            writer.add_summary(acc,epoch * batch_count + i)
        acc,loss=sess.run([test_accuracy,test_loss],feed_dict={x:testX,y_:testY,pkeep:test_keep})
        writer.add_summary(loss,epoch * batch_count + i)
        writer.add_summary(acc,epoch * batch_count + i)
        
        if epoch%5==0:
            print('epoch',epoch)
            print('test_accuracy',accuracy.eval(feed_dict={x:testX,y_:testY,pkeep:test_keep}))
        
    saver.save(sess, logs_path + '/' + model_name)  
            

epoch 0
test_accuracy 0.9501
epoch 5
test_accuracy 0.9762
epoch 10
test_accuracy 0.9755
epoch 15
test_accuracy 0.9767
epoch 20
test_accuracy 0.9787
epoch 25
test_accuracy 0.9801
epoch 30
test_accuracy 0.9809
epoch 35
test_accuracy 0.9814
epoch 40
test_accuracy 0.9808
epoch 45
test_accuracy 0.9787
epoch 50
test_accuracy 0.9764
epoch 55
test_accuracy 0.9766
